In [ ]:
#r "nuget: FsToolkit.ErrorHandling, 4.16.0"

In [ ]:
open System
open FsToolkit.ErrorHandling

Let's see an example of the result data type which can have either of the two values (this is also present in Scala as Either, however *in Scala, the left part is an error, whereas the Right part is Success*). **In F#, the left part is Ok, whereas the right part is Error**.

In [ ]:
let divide nr dr = 
  if (dr = 0) then Error("Cannot divide by 0.")
  else Ok(nr / dr)

The type of the divide function is `int -> int -> Result<int, string>`.

In [ ]:
let numerator = 4
let denominator = 2

let okResult = divide numerator denominator
let errorResult = divide numerator 0

Below is how we pattern match on a result value.

In [ ]:
match okResult with
  | Ok(result) -> printfn $"The result of dividing {numerator} with {denominator} is: {result}"
  | Error(e) -> printfn $"Error: {e}"

In [ ]:
match errorResult with
  | Ok(result) -> printfn $"The result of dividing {numerator} with {denominator} is: {result}"
  | Error(e) -> printfn $"Error: {e}"

Careful use of Result values also helps us to reduce / eliminate NPEs in the code. **Not just that, it also helps us communicate the cause of the problems.**

### Again, you might say that this pattern matching is not a big deal. We could've used the trusted old if-else statement?

First, let's see how to get the value of a result safely.

In [ ]:
let okVal = Result.defaultValue 0 okResult
printfn $"okVal: {okVal}"

Or in an idiomatic F# way -

In [ ]:
let iokVal = okResult |> Result.defaultValue 0
printfn $"iokVal: {iokVal}"

Now, let's see how we can use multiple result values together. Again, this is something which is not possible in C#. But `FsToolkit.ErrorHandling` makes it a walk in the park.

In [ ]:
let printResult result =
    match result with
    | Ok i -> printfn $"The result is: {i}"
    | Error e -> printfn $"Error: {e}"

let printResultWithErrors (result: Result<'a list, 'b list>) =
    match result with
    | Ok is -> printfn $"The result is: {is}"
    | Error es -> printfn $"Errors: {es}"

In [ ]:
let tryParseInt (str: string) =
    match Int32.TryParse str with
    | (true, i) -> Ok i
    | _ -> Error $"Could not parse string {str} to int."

In [ ]:
let addAllOks = result {
    let! x = tryParseInt "7"
    let! y = tryParseInt "5"
    let! z = tryParseInt "2"
    return x + y + z
}

printResult addAllOks

### Equivalent Rust code

```rust
use std::num::ParseIntError;

fn try_parse_int(s: &str) -> Result<i32, ParseIntError> {
    s.parse::<i32>()
}

fn add_all_oks() -> Result<i32, ParseIntError> {
    let x = try_parse_int("7")?; // early return if None
    let y = try_parse_int("5")?; // early return if None
    let z = try_parse_int("2")?; // early return if None
    Ok(x + y + z)
}
```

In [ ]:
let addOksWithError = result {
    let! x = tryParseInt "7"
    let! y = tryParseInt "sad"
    let! z = tryParseInt "2"
    return x + y + z
}

printResult addOksWithError

If you have a list of string values, and you use the above function, you get a list of result values.

In [ ]:
let listOfResults =
    ["1"; "2"; "3"; "cannotParse"]
    |> List.map tryParseInt

printfn $"listOfResults (which is of type list<Result<int, string>>):"
listOfResults |> List.iter printResult

But what if you want a Result<list<int>, string>? Simple, use `sequenceResultM` method.

In [ ]:
let resultList = 
    ["1"; "2"; "3"]
    |> List.map tryParseInt
    |> List.sequenceResultM

printfn $"resultList (which is of type Result<list<int>, string>):"
printResult resultList

But what if one of the values is an error?

In [ ]:
let resultList = 
    ["1"; "foo"; "3"; "bar"]
    |> List.map tryParseInt
    |> List.sequenceResultM

printfn $"resultList (which is of type Result<list<int>, string>):"
printResult resultList

*No, but we don't want that*. **We want to see all the errors**. That's where `sequenceResultA` method comes into picture.

#### You might have guessed that the `M` stands for `Monad` whereas the `A` stands for `Applicative`.

In [ ]:
let resultListWithAllErrors = 
    ["1"; "foo"; "3"; "bar"]
    |> List.map tryParseInt
    |> List.sequenceResultA

printfn $"resultListWithAllErrors (which is of type Result<list<int>, list<string>>):"
printResultWithErrors resultListWithAllErrors